# Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import TweedieRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from scipy.stats import randint, uniform, loguniform
from datetime import timedelta
from itertools import product
import holidays

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Lendo Bases

In [4]:
df_train = pd.read_csv('C:\\Users\\joaov\\OneDrive\\Área de Trabalho\\Python\\lev-challenge\\data\\processed\\train_dataset.csv')
df_test = pd.read_csv('C:\\Users\\joaov\\OneDrive\\Área de Trabalho\\Python\\lev-challenge\\data\\processed\\test_dataset.csv')

In [4]:
df_train

,date,bank,product,production,weekday,mean_bank,mean_product,mean_bp,std_bank,std_product,std_bp,lag_1,lag_7,lag_30,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_std_7,rolling_std_30,rolling_std_60,day_of_week,month,is_month_end,fortnight,is_weekend,is_holiday,is_pre_holiday,is_post_holiday,diff_mean_lag1_bank,diff_mean_lag7_bank,diff_mean_lag30_bank,diff_mean_lag1_product,diff_mean_lag7_product,diff_mean_lag30_product,diff_mean_lag1_bank_product,diff_mean_lag7_bank_product,diff_mean_lag30_bank_product,ratio_mean_lag1_bank,ratio_mean_lag7_bank,ratio_mean_lag30_bank,ratio_mean_lag1_product,ratio_mean_lag7_product,ratio_mean_lag30_product,ratio_mean_lag1_bank_product,ratio_mean_lag7_bank_product,ratio_mean_lag30_bank_product,trend
0,2025-01-01,28,9464,21319.95,Wednesday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2025-01-01,62,17488,805.26,Wednesday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2025-01-01,72,41047,184.54,Wednesday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,2025-01-01,72,64447,479.09,Wednesday,184.540000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,2025-01-01,97,17488,1363.57,Wednesday,0.000000,805.260000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60390,2025-07-05,291,91226,2327.30,Saturday,38962.390268,5472.918182,5472.918182,124303.428451,4663.514896,4663.514896,4029.90,9706.57,161.67,5489.994286,5926.932667,5853.421667,3162.292567,4520.968827,4587.821104,5,7,0,First Fortnight,1,0,0,0,-34932.490268,-29255.820268,-38800.720268,-1443.018182,4233.651818,-5311.248182,-1443.018182,4233.651818,-5311.248182,0.103431,0.249127,0.004149,0.736335,1.773564,0.029540,0.736335,1.773564,0.029540,77
60391,2025-07-05,291,91227,585.07,Saturday,38928.594059,2930.330435,2930.330435,124251.009188,2631.597680,2631.597680,941.65,612.06,2453.19,2844.184286,2498.675000,2764.559167,2590.764216,2269.005821,2498.964993,5,7,0,First Fortnight,1,0,0,0,-37986.944059,-38316.534059,-36475.404059,-1988.680435,-2318.270435,-477.140435,-1988.680435,-2318.270435,-477.140435,0.024189,0.015723,0.063018,0.321346,0.208871,0.837172,0.321346,0.208871,0.837172,69
60392,2025-07-05,291,91229,375.54,Saturday,38893.254406,2784.363521,2784.363521,124199.139871,3621.817049,3621.817049,554.33,2465.85,2030.63,2902.705714,2374.367333,2910.632667,2254.435884,3159.192641,3763.939062,5,7,0,First Fortnight,1,0,0,0,-38338.924406,-36427.404406,-36862.624406,-2230.033521,-318.513521,-753.733521,-2230.033521,-318.513521,-753.733521,0.014253,0.063400,0.05221

In [5]:
df_test

,date,bank,product,production,weekday,mean_bank,mean_product,mean_bp,std_bank,std_product,std_bp,lag_1,lag_7,lag_30,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_std_7,rolling_std_30,rolling_std_60,day_of_week,month,is_month_end,fortnight,is_weekend,is_holiday,is_pre_holiday,is_post_holiday,diff_mean_lag1_bank,diff_mean_lag7_bank,diff_mean_lag30_bank,diff_mean_lag1_product,diff_mean_lag7_product,diff_mean_lag30_product,diff_mean_lag1_bank_product,diff_mean_lag7_bank_product,diff_mean_lag30_bank_product,ratio_mean_lag1_bank,ratio_mean_lag7_bank,ratio_mean_lag30_bank,ratio_mean_lag1_product,ratio_mean_lag7_product,ratio_mean_lag30_product,ratio_mean_lag1_bank_product,ratio_mean_lag7_bank_product,ratio_mean_lag30_bank_product,trend
0,2025-07-06,53,28084,1486.58,Sunday,22465.268825,19074.300331,19074.300331,73399.532641,20829.719935,20829.719935,1486.58,6730.50,4.496360e+03,6502.377143,6306.711667,15084.125500,5876.657141,5374.933434,18815.338794,6,7,0,First Fortnight,1,0,0,0,-20978.688825,-15734.768825,-1.796891e+04,-17587.720331,-12343.800331,-14577.940331,-17587.720331,-12343.800331,-14577.940331,0.066172,0.299596,0.200147,0.077936,0.352857,0.235729,0.077936,0.352857,0.235729,121
1,2025-07-06,53,30162,30430.96,Sunday,22460.859685,6802.914286,6802.914286,73392.447501,14748.820338,14748.820338,60510.62,2116.06,6.285710e+03,21710.471429,7793.094667,6802.914286,31894.438356,17038.974719,14748.820338,6,7,0,First Fortnight,1,0,0,0,38049.760315,-20344.799685,-1.617515e+04,53707.705714,-4686.854286,-517.204286,53707.705714,-4686.854286,-517.204286,2.694047,0.094211,0.279852,8.894809,0.311052,0.923973,8.894809,0.311052,0.923973,42
2,2025-07-06,53,30164,3719.97,Sunday,22462.534427,4191.792593,4191.792593,73384.825509,5419.558371,5419.558371,6315.90,190.57,2.436440e+03,5709.081429,3751.298000,4191.792593,9705.383212,6198.001526,5419.558371,6,7,0,First Fortnight,1,0,0,0,-16146.634427,-22271.964427,-2.002609e+04,2124.107407,-4001.222593,-1755.352593,2124.107407,-4001.222593,-1755.352593,0.281175,0.008484,0.108467,1.506730,0.045463,0.581241,1.506730,0.045463,0.581241,54
3,2025-07-06,53,64416,334.50,Sunday,22458.596914,1422.451111,1422.451111,73377.617868,2151.213639,2151.213639,334.50,341.11,1.422451e+03,2224.740000,1422.451111,1422.451111,3096.070593,2151.213639,2151.213639,6,7,0,First Fortnight,1,0,0,0,-22124.096914,-22117.486914,-2.103615e+04,-1087.951111,-1081.341111,0.000000,-1087.951111,-1081.341111,0.000000,0.014894,0.015188,0.063337,0.235157,0.239804,1.000000,0.235157,0.239804,1.000000,18
4,2025-07-06,62,3139,0.00,Sunday,29680.778513,0.000000,0.000000,59993.713839,0.000000,0.000000,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6,7,0,First Fortnight,1,0,0,0,-29680.778513,-29680.778513,-2.968078e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26968,2025-09-23,282,63830,26221.88,Tuesday,356882.783586,531319.642358,531319.642358,576646.100070,623809.071689,623809.071689,383913.48,47275.16,1.416398e+06,502126.785714,766140.207667,698173.770333,273109.157931,418384.117266,358848.458821,1,9,0,Second Fortnight,0,0,0,0,27030.696414,-309607.623586,1.059516e+06,-147406.162358,-484044.482358,885078.777642,-147406.162358,-484044.482358,885078.777642,1.075741,0.132467,3.968806,0.722566,0.088977,2.665812,0.722566,0.088977,2.665812,212
26969,2025-09-23,291,91219,99.17,Tuesday,37568.634920,1806.807762,1806.807762,119405.774278,2613.379541,2613.379541,2155.83,1137.08,1.140420e+03,770.178571,1558.094333,1751.140333,735.253447,2183.024620,2184.186430,1,9,0,Second Fortnight,0,0,0,0,-35412.804920,-36431.554920,-3.642821e+04,349.022238,-669.727762,-666.387762,349.022238,-669.727762,-666.387762,0.057384,0.030267,0.030356,1.193171,0.6293

# Pre-processamento

In [6]:
df_train.columns

Index(['date', 'bank', 'product', 'production', 'weekday', 'mean_bank',
       'mean_product', 'mean_bp', 'std_bank', 'std_product', 'std_bp', 'lag_1',
       'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30',
       'rolling_mean_60', 'rolling_std_7', 'rolling_std_30', 'rolling_std_60',
       'day_of_week', 'month', 'is_month_end', 'fortnight', 'is_weekend',
       'is_holiday', 'is_pre_holiday', 'is_post_holiday',
       'diff_mean_lag1_bank', 'diff_mean_lag7_bank', 'diff_mean_lag30_bank',
       'diff_mean_lag1_product', 'diff_mean_lag7_product',
       'diff_mean_lag30_product', 'diff_mean_lag1_bank_product',
       'diff_mean_lag7_bank_product', 'diff_mean_lag30_bank_product',
       'ratio_mean_lag1_bank', 'ratio_mean_lag7_bank', 'ratio_mean_lag30_bank',
       'ratio_mean_lag1_product', 'ratio_mean_lag7_product',
       'ratio_mean_lag30_product', 'ratio_mean_lag1_bank_product',
       'ratio_mean_lag7_bank_product', 'ratio_mean_lag30_bank_product',
       'trend'],
      

## Ordenando Base
Processo essencial quando tratamos de séries temporais para evitar data leakage

In [5]:
df_train.sort_values(by=['date', 'bank', 'product'], inplace=True) 
df_test.sort_values(by=['date', 'bank', 'product'], inplace=True)

## Rescaling do Alvo
Como visto no notebook anterior, a escala do alvo pode prejudicar bastante a assertividade do modelo

In [6]:
df_train['production'] = df_train['production'].clip(lower=0)
df_test['production'] = df_test['production'].clip(lower=0)
df_train['production_log'] = np.log1p(df_train['production']) # log(1+Y)
df_test['production_log'] = np.log1p(df_test['production']) # log(1+Y)

## Flag de produção ativa

In [7]:
df_train['is_active'] = (df_train['production'] > 0).astype(int)
df_test['is_active'] = (df_test['production'] > 0).astype(int)

In [8]:
category_cols = ['bank', 'product', 'day_of_week', 'month', 'fortnight', 'weekday']
df_train[category_cols] = df_train[category_cols].astype('category')
df_test[category_cols] = df_test[category_cols].astype('category')

In [9]:
X_train_clf = df_train.drop(['date', 'production', 'production_log', 'is_active'], axis=1)
y_train_clf = df_train['is_active']
X_test_clf = df_test.drop(['date', 'production', 'production_log', 'is_active'], axis=1)
y_test_clf = df_test['is_active']

# Pipelines de Processamento

In [10]:
numerical_cols = X_train_clf.drop(category_cols, axis=1).columns.tolist()
categorical_cols = X_train_clf[category_cols].columns.tolist()

# Pipeline para colunas numéricas
num_transf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', MinMaxScaler())
])

# Pipeline para colunas categóricas
cat_transf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('encoder', CatBoostEncoder(handle_missing='value', handle_unknown='value')),
    ('scaler', MinMaxScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transf, numerical_cols),
        ('cat', cat_transf, categorical_cols)
    ],
    remainder='drop'
)


# Tuning de Hiperparâmetros
Aqui irei testar duas abordagens diferentes para tentar solucionar o problema de uma base sparse e com inflação de zeros

## Método 1: Two-staged model
Utilização de um modelo de classificação de produção ativa (0 ou 1) e para as series previstas como classificação ativa, prever a produção

### Classifier

In [14]:
# Modelos
xgb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(eval_metric='auc', use_label_encoder=False, random_state=42)) 
])

rf_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

models = {
    'xgb': xgb_pipe,
    'rf': rf_pipe
}

# Hiperparâmetros
params = {
    'xgb': {
        'classifier__n_estimators': randint(100, 500), # Antes: Integer(100, 500)
        'classifier__max_depth': randint(3, 20),
        'classifier__learning_rate': uniform(0.01, 0.29), # Antes: Real(0.01, 0.3)
        'classifier__subsample': uniform(0.5, 1.0) # Antes: Real(0.5, 1.0)
    },
    'rf': {
        'classifier__n_estimators': randint(100, 500),
        'classifier__max_depth': randint(3, 20),
        'classifier__min_samples_split': randint(2, 10),
        'classifier__min_samples_leaf': randint(1, 10)
    }
}


# Métricas
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='binary', zero_division=0),
    'recall': make_scorer(recall_score, average='binary', zero_division=0),
    'f1_score': make_scorer(f1_score, average='binary', zero_division=0),
}

# Validação Temporal
cv = TimeSeriesSplit(n_splits=3)

# Armazenar resultados
grids = {}

# Treinamento e Avaliação
for model_name, model in models.items():
    print(f"\n{'='*50}")
    print(f"TREINANDO CLASSIFICADOR {model_name.upper()}")
    print(f"{'='*50}")
    
    grids[model_name] = RandomizedSearchCV(
        estimator=model,
        param_distributions=params[model_name],
        cv=cv,
        scoring=scoring,
        refit='precision',
        n_jobs=-1,
        verbose=0,
        n_iter=10, 
        random_state=42
    )
    
    grids[model_name].fit(X_train_clf, y_train_clf) 
    
    # Resultados
    best_model = grids[model_name].best_estimator_
    best_index = grids[model_name].best_index_
    results = grids[model_name].cv_results_

    print(f"\nMelhores resultados para {model_name}:")
    print(f"Melhor combinação de parâmetros: {grids[model_name].best_params_}")

    for score in scoring:
        mean_score = results[f'mean_test_{score}'][best_index]
        std_score = results[f'std_test_{score}'][best_index]
        print(f"  {score.capitalize()}: Média = {mean_score:.4f}, Desvio Padrão = {std_score:.4f}")


TREINANDO CLASSIFICADOR XGB


c:\Users\joaov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\joaov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\joaov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\joaov\AppData


Melhores resultados para xgb:
Melhor combinação de parâmetros: {'classifier__learning_rate': np.float64(0.05143137719736283), 'classifier__max_depth': 5, 'classifier__n_estimators': 249, 'classifier__subsample': np.float64(0.5564115790271003)}
  Accuracy: Média = 0.9764, Desvio Padrão = 0.0024
  Precision: Média = 0.9827, Desvio Padrão = 0.0026
  Recall: Média = 0.9928, Desvio Padrão = 0.0034
  F1_score: Média = 0.9877, Desvio Padrão = 0.0013

TREINANDO CLASSIFICADOR RF

Melhores resultados para rf:
Melhor combinação de parâmetros: {'classifier__max_depth': 13, 'classifier__min_samples_leaf': 8, 'classifier__min_samples_split': 6, 'classifier__n_estimators': 199}
  Accuracy: Média = 0.9778, Desvio Padrão = 0.0025
  Precision: Média = 0.9826, Desvio Padrão = 0.0027
  Recall: Média = 0.9943, Desvio Padrão = 0.0035
  F1_score: Média = 0.9884, Desvio Padrão = 0.0014


**O modelo de classificação tem um desempenho ótimo (que já é esperado), o desafio maior é o modlo de regressão**

In [15]:
model_clf = best_model
df_train['IS_ACTIVE_PRED'] = model_clf.predict(X_train_clf)
df_test['IS_ACTIVE_PRED'] = model_clf.predict(X_test_clf)

### Regressão

In [16]:
X_train_reg = df_train[df_train['IS_ACTIVE_PRED'] == 1].drop(['date', 'production', 'production_log', 'is_active', 'IS_ACTIVE_PRED'], axis=1)
y_train_reg = df_train[df_train['IS_ACTIVE_PRED'] == 1]['production_log']
X_test_reg = df_test[df_test['IS_ACTIVE_PRED'] == 1].drop(['date', 'production', 'production_log', 'is_active', 'IS_ACTIVE_PRED'], axis=1)
y_test_reg = df_test[df_test['IS_ACTIVE_PRED'] == 1]['production_log']

In [20]:
def rmse_score(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Pipelines
xgb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42)) 
])
rf_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Modelos
models = {
    'xgb': xgb_pipe,
    'rf': rf_pipe
}

# Hiperparâmetros
params = {
    'xgb': {
        'regressor__n_estimators': randint(100, 500),
        'regressor__max_depth': randint(3, 20),
        'regressor__learning_rate': uniform(0.01, 0.29),
        'regressor__subsample': uniform(0.5, 1)
    },
    'rf': {
        'regressor__n_estimators': randint(100, 500),
        'regressor__max_depth': randint(3, 20),
        'regressor__min_samples_split': randint(2, 10),
        'regressor__min_samples_leaf': randint(1, 10)
    }
}

# Scoring e CV
scoring = {
    'neg_mae': 'neg_mean_absolute_error',
    'neg_medae': 'neg_median_absolute_error',
    'neg_rmse': 'neg_mean_squared_error',
    'r2': 'r2',
}

cv = TimeSeriesSplit(n_splits=3)
grids = {}

# Treinar modelos
for model_name, model in models.items():
    print(f"\n{'='*50}")
    print(f"TREINANDO {model_name.upper()}")
    print(f"{'='*50}")

    grids[model_name] = RandomizedSearchCV(
        estimator=model,
        param_distributions=params[model_name],
        cv=cv,
        scoring=scoring,
        refit='neg_medae', 
        n_jobs=-1,
        verbose=0,
        n_iter=10, 
        random_state=42
    )
    
    grids[model_name].fit(X_train_reg, y_train_reg)
    
    # Resultados
    best_model_reg = grids[model_name].best_estimator_
    best_index_reg = grids[model_name].best_index_
    results_reg = grids[model_name].cv_results_
    
    print(f"Melhores parâmetros: {grids[model_name].best_params_}")
    
    # Métricas no treino
    for score in scoring:
        if score == 'neg_mape':
            mean_score = -results_reg[f'mean_test_{score}'][best_index_reg]
            std_score = results_reg[f'std_test_{score}'][best_index_reg]
            print(f"  MAPE (CV): {mean_score:.4f} (±{std_score:.4f})")
        else:
            mean_score = -results_reg[f'mean_test_{score}'][best_index_reg]
            std_score = results_reg[f'std_test_{score}'][best_index_reg]
            print(f"  {score.upper()}: {mean_score:.4f} (±{std_score:.4f})")

    # RMSE manual
    train_pred = best_model_reg.predict(X_train_reg)
    print(f"  RMSE (Log Scale): {rmse_score(y_train_reg, train_pred):.4f}")

# Comparação final no teste
def compare_models(grids, X_test, y_test):
    print(f"\n{'='*50}")
    print("COMPARAÇÃO - CONJUNTO DE TESTE")
    print(f"{'='*50}")
    
    results = {}
    
    for model_name, grid in grids.items():
        best_model_reg = grid.best_estimator_
        y_pred = best_model_reg.predict(X_test_reg)

        medae = median_absolute_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = rmse_score(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[model_name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MEDAE': medae
        }
        
        print(f"\n {model_name.upper()}:")
        print(f"   MAE:   {mae:.4f}")
        print(f"   RMSE:  {rmse:.4f}")
        print(f"   R²:    {r2:.4f}")
        print(f"   MEDAE: {medae:.4f}")

    return results


TREINANDO XGB


c:\Users\joaov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\joaov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\joaov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\joaov\AppData

Melhores parâmetros: {'regressor__learning_rate': np.float64(0.05143137719736283), 'regressor__max_depth': 5, 'regressor__n_estimators': 249, 'regressor__subsample': np.float64(0.5564115790271003)}
  NEG_MAE: 0.9911 (±0.0430)
  NEG_MEDAE: 0.7173 (±0.0345)
  NEG_RMSE: 2.1694 (±0.1958)
  R2: -0.5036 (±0.0426)
  RMSE (Log Scale): 1.3272

TREINANDO RF
Melhores parâmetros: {'regressor__max_depth': 13, 'regressor__min_samples_leaf': 8, 'regressor__min_samples_split': 6, 'regressor__n_estimators': 199}
  NEG_MAE: 0.9854 (±0.0400)
  NEG_MEDAE: 0.7148 (±0.0367)
  NEG_RMSE: 2.1557 (±0.1829)
  R2: -0.5068 (±0.0391)
  RMSE (Log Scale): 1.2960


## Método 2: Tweedie Regressor
O Tweedie é um algoritmo que generaliza as distribuições Poisson e Gamma, muito utilizado com dados assimétricos, que é o nosso caso

In [ ]:
X_train_tweedie = df_train.drop(['date', 'production', 'production_log', 'is_active', 'IS_ACTIVE_PRED'], axis=1)
y_train_tweedie = df_train['production_log']

In [13]:
tweedie_model = TweedieRegressor(link='log')

# Espaço de busca focado na distribuição e regularização
tweedie_params = {
    'regressor__power': uniform(1.1, 0.8), # Busca entre 1.1 e 1.9
    'regressor__alpha': loguniform(1e-5, 1.0) # Busca em escala logarítmica
}

# Pipeline
tweedie_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', tweedie_model) 
])


# Métricas
scoring = {
    'neg_mae': 'neg_mean_absolute_error',
    'neg_rmse': 'neg_mean_squared_error',
    'r2': 'r2',
    'neg_medae': 'neg_median_absolute_error'
}

# CV (Time Series Split)
cv = TimeSeriesSplit(n_splits=3)

tweedie_grid = RandomizedSearchCV(
    estimator=tweedie_pipe,
    param_distributions=tweedie_params,
    cv=cv,
    scoring=scoring,
    refit='neg_mae', # Otimizar por MAE (ou mude para neg_medae, se preferir robustez)
    n_jobs=-1,
    verbose=0,
    n_iter=10, 
    random_state=42
)

tweedie_grid.fit(X_train_tweedie, y_train_tweedie) 

# Resultados

best_model = tweedie_grid.best_estimator_
best_index = tweedie_grid.best_index_
results = tweedie_grid.cv_results_

print(f"\n{'='*50}")
print("RESULTADOS TWEEDIE REGRESSOR")
print(f"{'='*50}")
print(f"Melhores parâmetros: {tweedie_grid.best_params_}")

for score in scoring:
    if score == 'neg_mape':
        mean_score = -results[f'mean_test_{score}'][best_index]
        std_score = results[f'std_test_{score}'][best_index]
        print(f"  MAPE (CV): {mean_score:.4f} (±{std_score:.4f})")
    else:
        # Reverte o sinal para as métricas negativas
        mean_score = -results[f'mean_test_{score}'][best_index] if score.startswith('neg_') else results[f'mean_test_{score}'][best_index]
        std_score = results[f'std_test_{score}'][best_index]
        print(f"  {score.upper()}: {mean_score:.4f} (±{std_score:.4f})")


RESULTADOS TWEEDIE REGRESSOR
Melhores parâmetros: {'regressor__alpha': np.float64(0.010129197956845729), 'regressor__power': np.float64(1.6664580622368366)}
  NEG_MAE: 1.2860 (±0.0632)
  NEG_RMSE: 3.5161 (±0.2170)
  R2: 0.4747 (±0.0236)
  NEG_MEDAE: 0.9197 (±0.0557)


## Base de Teste

In [14]:
tweedie_model = best_model
df_test['PRODUCTION_PRED_LOG'] = tweedie_model.predict(df_test)
df_test['PRODUCTION_PRED'] = np.expm1(df_test['PRODUCTION_PRED_LOG'])
mae = mean_absolute_error(df_test['production_log'], df_test['PRODUCTION_PRED_LOG'])
rmse = root_mean_squared_error(df_test['production_log'], df_test['PRODUCTION_PRED_LOG'])
r2 = r2_score(df_test['production_log'], df_test['PRODUCTION_PRED_LOG'])
medae = median_absolute_error(df_test['production_log'], df_test['PRODUCTION_PRED_LOG'])
print(f"\n{'='*50}")
print("AVALIAÇÃO FINAL NO TESTE - TWEEDIE REGRESSOR")
print(f"{'='*50}")
print(f"  MAE:   {mae:.4f}")
print(f"  RMSE:  {rmse:.4f}")
print(f"  R²:    {r2:.4f}")
print(f"  MEDAE: {medae:.4f}")


AVALIAÇÃO FINAL NO TESTE - TWEEDIE REGRESSOR
  MAE:   1.2956
  RMSE:  1.9533
  R²:    0.3966
  MEDAE: 0.9257


In [46]:
import joblib
joblib.dump(tweedie_model, "model_tweedie.pkl")

['model_tweedie.pkl']

# Conclusão
Como podemos observar o Tweedie Regressor foi o algoritmo que trouxe maior estabilidade e menores erros, com o R2 em 47% é um bom passo para métricas observadas nos algoritmos de árvore anteriores (com R2 negativos, piores que o aleatório)

# Base de Retorno (oot)

In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
df['date'] = pd.to_datetime(df['date'])
df = df.drop(columns=['PRODUCTION_PRED_LOG', 'PRODUCTION_PRED', 'is_active', 'production_log'], axis=1)
df.sort_values(by=['date', 'bank', 'product'], inplace=True)
df

,date,bank,product,production,weekday,mean_bank,mean_product,mean_bp,std_bank,std_product,std_bp,lag_1,lag_7,lag_30,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_std_7,rolling_std_30,rolling_std_60,day_of_week,month,is_month_end,fortnight,is_weekend,is_holiday,is_pre_holiday,is_post_holiday,diff_mean_lag1_bank,diff_mean_lag7_bank,diff_mean_lag30_bank,diff_mean_lag1_product,diff_mean_lag7_product,diff_mean_lag30_product,diff_mean_lag1_bank_product,diff_mean_lag7_bank_product,diff_mean_lag30_bank_product,ratio_mean_lag1_bank,ratio_mean_lag7_bank,ratio_mean_lag30_bank,ratio_mean_lag1_product,ratio_mean_lag7_product,ratio_mean_lag30_product,ratio_mean_lag1_bank_product,ratio_mean_lag7_bank_product,ratio_mean_lag30_bank_product,trend
0,2025-01-01,28,9464,21319.95,Wednesday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2025-01-01,62,17488,805.26,Wednesday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2025-01-01,72,41047,184.54,Wednesday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,2025-01-01,72,64447,479.09,Wednesday,184.540000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,2025-01-01,97,17488,1363.57,Wednesday,0.000000,805.260000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87363,2025-09-23,282,63830,26221.88,Tuesday,356882.783586,531319.642358,531319.642358,576646.100070,623809.071689,623809.071689,383913.48,47275.16,1416398.42,502126.785714,766140.207667,698173.770333,273109.157931,418384.117266,358848.458821,1,9,0,Second Fortnight,0,0,0,0,27030.696414,-309607.623586,1.059516e+06,-147406.162358,-484044.482358,885078.777642,-147406.162358,-484044.482358,885078.777642,1.075741,0.132467,3.968806,0.722566,0.088977,2.665812,0.722566,0.088977,2.665812,212
87364,2025-09-23,291,91219,99.17,Tuesday,37568.634920,1806.807762,1806.807762,119405.774278,2613.379541,2613.379541,2155.83,1137.08,1140.42,770.178571,1558.094333,1751.140333,735.253447,2183.024620,2184.186430,1,9,0,Second Fortnight,0,0,0,0,-35412.804920,-36431.554920,-3.642821e+04,349.022238,-669.727762,-666.387762,349.022238,-669.727762,-666.387762,0.057384,0.030267,0.030356,1.193171,0.629331,0.631179,1.193171,0.629331,0.631179,143
87365,2025-09-23,291,91222,339.85,Tuesday,37551.002231,977.188085,977.188085,119380.429448,1086.695853,1086.695853,3717.93,2336.07,607.13,1699.924286,1571.338333,1564.693333,1072.811228,1270.151417,1309.580017,1,9,0,Second Fortnight,0,0,0,0,-33833.072231,-35214.932231,-3.694387e+04,2740.741915,1358.881915,-370.058085,2740.74191

In [29]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import product
import holidays
import gc

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================
MAX_LAG = 60
DAYS_INACTIVE_THRESHOLD = 90
FEATURE_COLS = X_train_tweedie.columns.tolist()

print("Iniciando previsão...")
print(f"Features do modelo: {len(FEATURE_COLS)}")

# ==============================================================================
# 1. FILTRAR PARES ATIVOS
# ==============================================================================
last_dates = df.groupby(['bank', 'product'])['date'].max().reset_index()
last_dates.columns = ['bank', 'product', 'last_date']

cutoff_date = df['date'].max() - timedelta(days=DAYS_INACTIVE_THRESHOLD)
active_pairs_df = last_dates[last_dates['last_date'] >= cutoff_date][['bank', 'product']]

print(f"\nPares ativos: {len(active_pairs_df)} de {len(last_dates)}")

# ==============================================================================
# 2. CRIAR GRID OOT SIMPLIFICADO
# ==============================================================================
future_dates = pd.date_range(
    start=df['date'].max() + timedelta(days=1), 
    periods=180, 
    freq='D'
)

# Criar apenas pares que existem (sem cartesiano)
oot_list = []
for _, row in active_pairs_df.iterrows():
    for date in future_dates:
        oot_list.append({
            'date': date,
            'bank': row['bank'],
            'product': row['product']
        })

df_oot = pd.DataFrame(oot_list)
print(f"Grid OOT criado: {len(df_oot)} linhas")

# ==============================================================================
# 3. FEATURES DE CALENDÁRIO (VETORIZADO)
# ==============================================================================
br_holidays = holidays.Brazil()
holiday_set = set(br_holidays)

df_oot['date'] = pd.to_datetime(df_oot['date'])
df_oot['day_of_week'] = df_oot['date'].dt.dayofweek.astype(np.int8)
df_oot['weekday'] = df_oot['date'].dt.dayofweek.astype(np.int8)
df_oot['month'] = df_oot['date'].dt.month.astype(np.int8)
df_oot['is_month_end'] = df_oot['date'].dt.is_month_end.astype(np.int8)
df_oot['is_weekend'] = (df_oot['date'].dt.dayofweek > 4).astype(np.int8)
df_oot['is_holiday'] = df_oot['date'].isin(holiday_set).astype(np.int8)
df_oot['fortnight'] = (df_oot['date'].dt.day <= 15).astype(np.int8)
df_oot['is_pre_holiday'] = (df_oot['date'] + timedelta(days=1)).isin(holiday_set).astype(np.int8)
df_oot['is_post_holiday'] = (df_oot['date'] - timedelta(days=1)).isin(holiday_set).astype(np.int8)

print("Features de calendário criadas")

# ==============================================================================
# 4. FUNÇÕES OTIMIZADAS (SEM PANDAS NO LOOP)
# ==============================================================================
def get_lag_value(values, idx, lag):
    """Retorna lag de forma segura"""
    pos = idx - lag
    return values[pos] if pos >= 0 else 0

def safe_mean(arr):
    """Média segura"""
    return np.mean(arr) if len(arr) > 0 else 0

def safe_std(arr):
    """Desvio padrão seguro"""
    if len(arr) < 2:
        return 0
    try:
        return np.std(arr)
    except:
        return 0

# ==============================================================================
# 5. LOOP PRINCIPAL (BATCH PROCESSING)
# ==============================================================================
print("\nIniciando previsões...")

results = []
active_pairs = active_pairs_df.values  # Converter para numpy array

for idx, (bank_name, product_name) in enumerate(active_pairs):
    if idx % 10 == 0:
        print(f"Progresso: {idx}/{len(active_pairs)} pares processados")
        gc.collect()  # Liberar memória
    
    # Seed: histórico recente (como numpy array)
    mask = (df['bank'] == bank_name) & (df['product'] == product_name)
    history = df[mask].sort_values('date').tail(MAX_LAG + 1)
    
    if len(history) == 0:
        continue
    
    prod_values = history['production'].values.astype(np.float32)
    
    # Features estáticas (constantes)
    static = np.zeros(6, dtype=np.float32)
    for i, col in enumerate(['mean_bank', 'mean_product', 'mean_bp', 
                              'std_bank', 'std_product', 'std_bp']):
        if col in history.columns:
            static[i] = history[col].iloc[-1]
    
    # Grid futuro do par
    pair_mask = (df_oot['bank'] == bank_name) & (df_oot['product'] == product_name)
    pair_data = df_oot[pair_mask].reset_index(drop=True)
    n_days = len(pair_data)
    
    predictions = np.zeros(n_days, dtype=np.float32)
    
    # Loop de previsão (otimizado)
    for day_idx in range(n_days):
        hist_len = len(prod_values)
        
        # LAGS
        lag_1 = prod_values[-1] if hist_len > 0 else 0
        lag_7 = get_lag_value(prod_values, hist_len - 1, 7)
        lag_30 = get_lag_value(prod_values, hist_len - 1, 30)
        
        # ROLLINGS
        recent = prod_values[-60:] if hist_len >= 60 else prod_values
        
        rm7 = safe_mean(recent[-7:]) if len(recent) >= 7 else 0
        rs7 = safe_std(recent[-7:]) if len(recent) >= 7 else 0
        rm30 = safe_mean(recent[-30:]) if len(recent) >= 30 else 0
        rs30 = safe_std(recent[-30:]) if len(recent) >= 30 else 0
        rm60 = safe_mean(recent)
        rs60 = safe_std(recent)
        
        # Montar features
        row_dict = {
            'bank': bank_name, 'product': product_name,
            'day_of_week': pair_data.loc[day_idx, 'day_of_week'],
            'weekday': pair_data.loc[day_idx, 'weekday'],
            'month': pair_data.loc[day_idx, 'month'],
            'is_month_end': pair_data.loc[day_idx, 'is_month_end'],
            'is_weekend': pair_data.loc[day_idx, 'is_weekend'],
            'is_holiday': pair_data.loc[day_idx, 'is_holiday'],
            'is_pre_holiday': pair_data.loc[day_idx, 'is_pre_holiday'],
            'is_post_holiday': pair_data.loc[day_idx, 'is_post_holiday'],
            'fortnight': pair_data.loc[day_idx, 'fortnight'],
            'lag_1': lag_1, 'lag_7': lag_7, 'lag_30': lag_30,
            'rolling_mean_7': rm7, 'rolling_std_7': rs7,
            'rolling_mean_30': rm30, 'rolling_std_30': rs30,
            'rolling_mean_60': rm60, 'rolling_std_60': rs60,
            'trend': day_idx,
            'mean_bank': static[0], 'mean_product': static[1], 'mean_bp': static[2],
            'std_bank': static[3], 'std_product': static[4], 'std_bp': static[5],
        }
        
        # DIFF e RATIO
        for lag_val, lag_name in [(lag_1, '1'), (lag_7, '7'), (lag_30, '30')]:
            row_dict[f'diff_mean_lag{lag_name}_bank'] = lag_val - static[0]
            row_dict[f'diff_mean_lag{lag_name}_product'] = lag_val - static[1]
            row_dict[f'diff_mean_lag{lag_name}_bank_product'] = lag_val - static[2]
            row_dict[f'ratio_mean_lag{lag_name}_bank'] = lag_val / static[0] if static[0] != 0 else 1e-06
            row_dict[f'ratio_mean_lag{lag_name}_product'] = lag_val / static[1] if static[1] != 0 else 1e-06
            row_dict[f'ratio_mean_lag{lag_name}_bank_product'] = lag_val / static[2] if static[2] != 0 else 1e-06

        # Previsão
        try:
            X = pd.DataFrame([row_dict])[FEATURE_COLS].fillna(0)
            y_log = tweedie_model.predict(X)[0]
            y_pred = np.expm1(y_log)
            y_pred = max(0, min(y_pred, np.percentile(prod_values, 99) * 2))
        except Exception as e:
            y_pred = 0.0
        
        predictions[day_idx] = y_pred
        prod_values = np.append(prod_values, y_pred)[-MAX_LAG-1:]
    
    # Armazenar resultados
    for day_idx in range(n_days):
        results.append({
            'date': pair_data.loc[day_idx, 'date'],
            'bank': bank_name,
            'product': product_name,
            'prediction': predictions[day_idx]
        })

# ==============================================================================
# 6. EXPORTAR
# ==============================================================================
print("\nConsolidando resultados...")
df_forecast = pd.DataFrame(results)
df_forecast['prediction_lo'] = (df_forecast['prediction'] * 0.80).clip(lower=0)
df_forecast['prediction_hi'] = df_forecast['prediction'] * 1.20

df_forecast.to_csv('forecast.csv', index=False)
print(f"Concluído! {len(df_forecast)} linhas salvas em forecast.csv")

Iniciando previsão...
Features do modelo: 45

Pares ativos: 2718 de 4932
Grid OOT criado: 489240 linhas
Features de calendário criadas

Iniciando previsões...
Progresso: 0/2718 pares processados
Progresso: 10/2718 pares processados
Progresso: 20/2718 pares processados
Progresso: 30/2718 pares processados
Progresso: 40/2718 pares processados
Progresso: 50/2718 pares processados
Progresso: 60/2718 pares processados
Progresso: 70/2718 pares processados
Progresso: 80/2718 pares processados


C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joao

Progresso: 90/2718 pares processados
Progresso: 100/2718 pares processados
Progresso: 110/2718 pares processados


C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joao

Progresso: 120/2718 pares processados
Progresso: 130/2718 pares processados
Progresso: 140/2718 pares processados
Progresso: 150/2718 pares processados
Progresso: 160/2718 pares processados
Progresso: 170/2718 pares processados
Progresso: 180/2718 pares processados
Progresso: 190/2718 pares processados
Progresso: 200/2718 pares processados
Progresso: 210/2718 pares processados
Progresso: 220/2718 pares processados
Progresso: 230/2718 pares processados
Progresso: 240/2718 pares processados
Progresso: 250/2718 pares processados
Progresso: 260/2718 pares processados
Progresso: 270/2718 pares processados
Progresso: 280/2718 pares processados
Progresso: 290/2718 pares processados
Progresso: 300/2718 pares processados
Progresso: 310/2718 pares processados
Progresso: 320/2718 pares processados
Progresso: 330/2718 pares processados
Progresso: 340/2718 pares processados
Progresso: 350/2718 pares processados


...

Progresso: 2540/2718 pares processados
Progresso: 2550/2718 pares processados
Progresso: 2560/2718 pares processados
Progresso: 2570/2718 pares processados
Progresso: 2580/2718 pares processados
Progresso: 2590/2718 pares processados


C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joao

Progresso: 2600/2718 pares processados
Progresso: 2610/2718 pares processados
Progresso: 2620/2718 pares processados


C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joaov\AppData\Local\Temp\ipykernel_5408\4005188662.py:180: RuntimeWarning: overflow encountered in expm1
  y_pred = np.expm1(y_log)
C:\Users\joao

Progresso: 2630/2718 pares processados
Progresso: 2640/2718 pares processados
Progresso: 2650/2718 pares processados
Progresso: 2660/2718 pares processados
Progresso: 2670/2718 pares processados
Progresso: 2680/2718 pares processados
Progresso: 2690/2718 pares processados
Progresso: 2700/2718 pares processados
Progresso: 2710/2718 pares processados

Consolidando resultados...
Concluído! 489240 linhas salvas em forecast.csv


In [30]:
df = pd.read_csv('C:\\Users\\joaov\\OneDrive\\Área de Trabalho\\Python\\lev-challenge\\forecast.csv')
df

,date,bank,product,prediction,prediction_lo,prediction_hi
0,2025-09-24,2,17488,2826.9304,2261.54440,3392.3167
1,2025-09-25,2,17488,2897.9617,2318.36940,3477.5542
2,2025-09-26,2,17488,2940.7380,2352.59060,3528.8857
3,2025-09-27,2,17488,1039.9717,831.97736,1247.9661
4,2025-09-28,2,17488,940.3431,752.27450,1128.4117
...,...,...,...,...,...,...
489235,2026-03-18,295,137333,2590.5378,2072.43040,3108.6455
489236,2026-03-19,295,137333,2587.2046,2069.76370,3104.6458
489237,2026-03-20,295,137333,2581.4620,2065.16970,3097.7544
489238,2026-03-21,295,137333,924.0182,739.21454,1108.8219
